# microSCHC quickstart

This notebook provides an overview of microSCHC core functionalities through a complete example.

- The first part covers the installation of the microSCHC library.
- The second part is a primer on SCHC for compression and microSCHC architecture.
- The third part shows the usage of microSCHC on a concrete example: compressing the IPv6/UDP/CoAP/LwM2M traffic between a thermostat and a leshan application server.


## 1. Installing microSCHC

### Using a Virtual Environment

It is highly recommended to use a virtual environment to manage dependencies and avoid conflicts with other Python packages. You can choose between `venv` (built into Python) or `conda` (Anaconda/Miniconda).

#### Option 1: Using venv

1. Create a virtual environment:
   ```bash
   python -m venv microschc-env
   ```

2. Activate the virtual environment:
   - On Windows:
     ```bash
     .\microschc-env\Scripts\activate
     ```
   - On macOS/Linux:
     ```bash
     source microschc-env/bin/activate
     ```

3. Install microschc:
   ```bash
   pip install 'microschc[extras]'
   ```

#### Option 2: Using conda

1. Create a conda environment:
   ```bash
   conda create -n microschc-env python=3.12
   ```

2. Activate the conda environment:
   ```bash
   conda activate microschc-env
   ```

3. Install microschc:
   ```bash
   pip install 'microschc[extras]'
   ```

### Notes

- This tutorial requires the extra packages to be installed, i.e. `python-pcapng`, to open the PCAPng traffic capture. The default installation does not ship with any dependency apart `backports.strenum` for older Python versions.
- For Python versions below 3.11, the `backports.strenum` package will be automatically installed as a dependency.
- For Python 3.11 and above, `StrEnum` is part of the standard library, so no additional dependencies are required.



## Understanding SCHC (Static Context Header Compression)

SCHC is a compression protocol designed to efficiently compress headers in constrained networks. Let's break down its key concepts step by step:

### 1. Static Context

At the heart of SCHC is a **static context** - a set of rules that both the compressor and decompressor must know in advance. Think of it as a "file cabinet" containing a collection of rules:
```
+------------------+
| Static Context   |
|  +------------+  |
|  |   Rule 1   |  |
|  +------------+  |
|  +------------+  |
|  |   Rule 2   |  |
|  +------------+  |
|  +------------+  |
|  |   Rule 3   |  |
|  +------------+  |
|  +------------+  |
|  |   Rule 4   |  |
|  +------------+  |
+------------------+
```

In microSCHC, this context is represented by the `Context` class:

```python
from microschc.rfc8724extras import Context


context = Context(
    id="my_context",
    description="Context for IoT device communication",
    interface_id="eth0",
    parser_id="ipv6_udp_coap",
    ruleset=[...]  # List of rules
)
```

### 2. Rules

Each rule in the context describes how to compress a specific type of packet. A rule is an ordered collection of field descriptors that specify how to handle each field in the header:

```
+------------------+
|      Rule 1      |
+------------------+
| Field Descriptor |
| Field Descriptor |
| Field Descriptor |
|       ...        |
+------------------+
```

In microSCHC, rules are represented by the `RuleDescriptor` class:

```python
from microschc.rfc8724 import RuleDescriptor, RuleNature

rule = RuleDescriptor(
    id=Buffer(content=b'\x01'),  # Rule ID
    nature=RuleNature.COMPRESSION,
    field_descriptors=[...]  # Ordered list of field descriptors
)
```

### 3. Field Descriptors

Each field descriptor specifies how to handle a particular field in the header. It contains several important attributes:

```
+-------+--+--+--+------------+-----------------+---------------+
|Field 1|FL|FP|DI|Target Value|Matching Operator|Comp/Decomp Act|
+-------+--+--+--+------------+-----------------+---------------+
```

Where:
- **FID**: Field ID (e.g., Source IP, Destination IP)
- **FL**: Field Length in bits
- **FP**: Field Position (0 for regular fields, >0 for repeating fields)
- **DI**: Direction Indicator (Up, Down, or Bidirectional)
- **Target Value**: The expected value of the field
- **Matching Operator**: How to compare the field value
- **Comp/Decomp Act**: What to do with this field during compression/decompression

In microSCHC, field descriptors are represented by the `RuleFieldDescriptor` class:

```python
from microschc.rfc8724 import RuleFieldDescriptor, DirectionIndicator
from microschc.protocol.ipv6 import IPv6Fields

field_descriptor = RuleFieldDescriptor(
    id=IPv6Fields.SRC_ADDRESS,  # Field ID
    length=128,                 # Field Length
    position=0,                 # Field Position
    direction=DirectionIndicator.BIDIRECTIONAL,  # Direction
    target_value=Buffer(content=b'\x20\x01\x0d\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01', length=128),
    matching_operator=MatchingOperator.EQUAL,
    compression_decompression_action=CompressionDecompressionAction.NOT_SENT
)
```

### 4. Matching Operators and Compression Actions

The behavior of each field is controlled by two key components:

#### Matching Operators
These determine how to compare the field value with the target value:
- `EQUAL`: Field must exactly match the target value
- `IGNORE`: Field value is not checked
- `MSB`: Field must match the most significant bits of the target value
- `MATCH_MAPPING`: Field must match one of the values in a mapping table

#### Compression/Decompression Actions
These determine what to do with the field during compression:
- `NOT_SENT`: Field is not sent (decompressor knows the value)
- `VALUE_SENT`: Field is sent as is
- `LSB`: Only the least significant bits are sent
- `MAPPING_SENT`: A mapped value is sent instead
- `COMPUTE`: Field value is computed from other fields


### 5. Target Values and Binary Buffers

Target values in SCHC are represented using the `TargetValue` class, which is either a `MatchMapping` or a `Buffer`.


`Buffer`, which is the core class provides a robust way to handle binary data at the bit level. This is crucial because:

1. **Bit-level Precision**: The same byte sequence can represent different values depending on their bit length
2. **Padding Control**: Fields can be left-padded or right-padded
3. **Bit-level Operations**: Support for shifting, masking, and other binary operations

Here's how to create and use buffers for target values:

```python
from microschc.binary.buffer import Buffer, Padding

# Create a buffer with explicit bit length
ipv6_address = Buffer(
    content=b'\x20\x01\x0d\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01',
    length=128,  # IPv6 address is 128 bits
    padding=Padding.LEFT  # Default padding
)

# Create a buffer for a 4-bit field
small_field = Buffer(
    content=b'\x06',  # Binary: 0110
    length=4,         # Only use 4 bits
    padding=Padding.LEFT
)

# Create a buffer for a 12-bit field
larger_field = Buffer(
    content=b'\x06',  # Binary: 0000 0000 0110
    length=12,        # Use 12 bits
    padding=Padding.LEFT
)
```

The `Buffer` class provides several useful features:

1. **Bit-level Slicing**: Access individual bits or ranges of bits
2. **Padding Control**: Choose between left or right padding
3. **Binary Operations**: Support for AND, OR, XOR, and shifting
4. **Concatenation**: Combine buffers using the `+` operator

For more advanced features like bit-level manipulation, padding adjustments, and binary operations, see the [Buffer Tutorial](buffer_tutorial.ipynb).

`MatchMapping` is a class that represents a mapping between binary values and indices and is used in conjunction with the MO/CDA `match-mapping/mapping-sent`.

This is useful for compressing fields that exhibits a limited number of predefined values for a given traffic.

Here's how to create a MatchMapping for an IPv6 address field that assumes two possible values and their corresponding short (compressed) values.

- `2001:db8::20`: which is replaced by a 2-bits field  of binary value `0b00`
- `2001:db8::21`: which is replaced by a 2-bits field  of binary value `0b01`

```python

from microschc.binary.buffer import Buffer, Padding, MatchMapping


MatchMapping(forward_mapping={
    Buffer(content=b"\x20\x01\x0d\xb8\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20", length=128):Buffer(content=b'\x00', length=2),
    Buffer(content=b"\x20\x01\x0d\xb8\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x21", length=128):Buffer(content=b'\x01', length=2)
}), 
```

## From Theory to Practice: Applying SCHC to IoT Communication

Now that we understand the core concepts of SCHC, let's see how we can apply them to a real-world IoT scenario. We'll be working with a thermostat that communicates with a Leshan application server using IPv6, UDP, and CoAP protocols.


In our scenario we consider a traffic captured between a thermostat (2001:db8:a::3) and an application server (2001:db8:a::20).

The protocol stack used for this communication is based on IPv6, UDP, CoAP for all datagrams. 

At the application level, two formats are being used : 

- Lightweight M2M TLV format
- CBOR encoding format

3 lw-M2M resources are described:

- a temperature sensor (resource ID `3303`)
- a humidity sensor (resource ID `3304`) 
- a setpoint (resource ID `3308`).


The following resource IDs of the temperature and humidity sensors are used in the dataset:

- sensor value (resource ID `5700`)
- min measured value (resource ID `5601`)
- max measured value (resource ID `5602`)
- min range value (resource ID `5603`)
- max range value (resource ID `5604`)
- sensor units (resource ID `5701`)

A reset of the min/max measured values is triggered using the resource ID `5605`.


The following resource IDs of the setpoint are used in the dataset:

- setpoint value (resource ID `5900`)
- sensor units (resource ID `5701`)
- color (resource ID `5706`)
- application type (resource ID `5750`)
- timestamp (resource ID `5518`)
- fractional timestamp (resource ID `6050`)

### Understanding the Communication Pattern

The thermostat scenario follows a typical IoT communication pattern:
1. The server subscribes with an observe request to a resource 
 whose value he wants to track (such as the temperature and humidity value, min and max).
2. The device periodically sends updates about temperature and humidity. The dataset includes notifications using two different formats: the first one has values in lwm2m+tlv and the second one in cbor. In addition, these messages feature an observe sequence number which starts from 1 and is incremented at each new packet. Initially, it is encoded on 1 byte, then when the value exceeds xff, it switches to 2 bytes, and so on. Notifications are usually sent without confirmation (NON) but once in a while they are sent as confirmed messages (CON). 
3. The server then sends an empty ACK (no token) with the same message ID as the confirmed message. 
3. The server occasionally sends commands to reset min/max values or update setpoints.

The dominant traffic is therefore from the device (client) to the server ( `2001:db8::3` -> `2001:db8::20`) and the most frequent packets are readings from sensors, i.e. CoAP code `2.05` (content) & resource ID `5700` (sensor value).
Notice the repeating CoAP Token values, e.g. `d159`, `2150`, `8d43`, `3709` and `1f0a`, each corresponding to an observe subscription, as described in Fig. 1.

![Fig 1. Observe traffic pattern](quickstart-files/observe-pattern.png)





### Identifying Compression Opportunities

Looking globally at our traffic, we can identify several opportunities for compression:

1. **IPv6 Headers**:
   - Version field is always 6
   - Source and destination addresses are fixed
   - Traffic class and flow label often have default values

2. **UDP Headers**:
   - Source and destination ports are consistent
   - Length can be computed from the payload
   - Checksum can be computed at the receiver

3. **CoAP Headers**:
   - Version is fixed
   - Message types follow patterns (CON, NON, ACK)
   - Token values repeat for observe subscriptions
   - Observe sequence numbers increment predictably

4. **LwM2M/CBOR Payloads**:
   - Resource IDs are consistent thoughout the dataset.

### From Global Opportunities to Specific Templates

While we've identified general compression opportunities across protocol layers, SCHC requires more precise matching. A rule must match an entire packet structure (template) to be applicable. This is because SCHC uses a "match-all" approach - either all fields in a rule match the packet, or the rule doesn't apply at all.

In our dataset, we've identified 6 distinct templates (ordered from the most common to the least), each representing a specific packet structure:

- template 0: `2001:db8::3` -> `2001:db8::20`, CoAP non-confirmable and confirmable (2.00 and 2.01), LwM2M notifications (resource ID `5700`)
- template 1: `2001:db8::3` -> `2001:db8::20`, CoAP non-confirmable and confirmable (2.00 and 2.01), CBOR notifications, token `1f0a`.
- template 2: `2001:db8::20` -> `2001:db8::3`, ACK (no CoAP Token).
- template 3: `2001:db8::20` -> `2001:db8::3`, CoAP POST --> reset min/max measured values (resource ID `5605`). 
- template 4: `2001:db8::3` -> `2001:db8::20`, ACK (with CoAP Token, CoAP code 2.04)
- template 5: `2001:db8::20` -> `2001:db8::3`, CoAP PUT, confirmable resource ID `5900` (setpoint setting).

To better understand microSCHC operation, let's focus on the first template (template 0), which features CoAP notifications (2.00, 2.01 status codes) with a LwM2M format. 

The following table shows its header fields and their values in binary and hexadecimal formats.


```
+----------------------------+-----------------------------------------------+
|          field ID          |                    length                     |
+============================+===============================================+
| 0-IPv6:Version             | 1 value of size: (4)                          |
|                            | [----0110 ](4): 8544                          |
+----------------------------+-----------------------------------------------+
| 1-IPv6:Traffic Class       | 1 value of size: (8)                          |
|                            | [00000000](8): 8544                           |
+----------------------------+-----------------------------------------------+
| 2-IPv6:Flow Label          | 1 value of size: (20)                         |
|                            | [0ff85f](20): 8544                            |
+----------------------------+-----------------------------------------------+
| 3-IPv6:Payload Length      | 7 values of sizes: (16)                       |
|                            | [00000000 00100001](16): 5747                 |
|                            | [00000000 00011101](16): 2052                 |
|                            | [00000000 00010111](16): 532                  |
|                            | [00000000 00100000](16): 154                  |
|                            | [00000000 00011100](16): 47                   |
|                            | [00000000 00010110](16): 9                    |
|                            | [00000000 00011001](16): 3                    |
+----------------------------+-----------------------------------------------+
| 4-IPv6:Next Header         | 1 value of size: (8)                          |
|                            | [00010001](8): 8544                           |
+----------------------------+-----------------------------------------------+
| 5-IPv6:Hop Limit           | 1 value of size: (8)                          |
|                            | [01000000](8): 8544                           |
+----------------------------+-----------------------------------------------+
| 6-IPv6:Source Address      | 1 value of size: (128)                        |
|                            | [20010db8000a00000000000000000003](128): 8544 |
+----------------------------+-----------------------------------------------+
| 7-IPv6:Destination Address | 1 value of size: (128)                        |
|                            | [20010db8000a00000000000000000020](128): 8544 |
+----------------------------+-----------------------------------------------+
| 8-UDP:Source Port          | 1 value of size: (16)                         |
|                            | [10010000 10100000](16): 8544                 |
+----------------------------+-----------------------------------------------+
| 9-UDP:Destination Port     | 1 value of size: (16)                         |
|                            | [00010110 00110011](16): 8544                 |
+----------------------------+-----------------------------------------------+
| 10-UDP:Length              | 7 values of sizes: (16)                       |
|                            | [00000000 00100001](16): 5747                 |
|                            | [00000000 00011101](16): 2052                 |
|                            | [00000000 00010111](16): 532                  |
|                            | [00000000 00100000](16): 154                  |
|                            | [00000000 00011100](16): 47                   |
|                            | [00000000 00010110](16): 9                    |
|                            | [00000000 00011001](16): 3                    |
+----------------------------+-----------------------------------------------+
| 11-UDP:Checksum            | 8009 values of sizes: (16)                    |
|                            | [01101100 01010010](16): 3                    |
|                            | [10010100 00101001](16): 3                    |
|                            | [00110110 01111000](16): 3                    |
|                            | [11010101 01000000](16): 3                    |
|                            | [00101111 00111000](16): 3                    |
|                            | [01011100 00100100](16): 3                    |
|                            | [01000111 00010000](16): 3                    |
|                            | [00001111 10000110](16): 3                    |
|                            | [00001010 10111011](16): 3                    |
|                            | [00100010 11111100](16): 3                    |
|                            | ...                                           |
+----------------------------+-----------------------------------------------+
| 12-CoAP:Version            | 1 value of size: (2)                          |
|                            | [------01 ](2): 8544                          |
+----------------------------+-----------------------------------------------+
| 13-CoAP:Type               | 2 values of sizes: (2)                        |
|                            | [------01 ](2): 8271                          |
|                            | [------00 ](2): 273                           |
+----------------------------+-----------------------------------------------+
| 14-CoAP:Token Length       | 1 value of size: (4)                          |
|                            | [----0010 ](4): 8544                          |
+----------------------------+-----------------------------------------------+
| 15-CoAP:Code               | 1 value of size: (8)                          |
|                            | [01000101](8): 8544                           |
+----------------------------+-----------------------------------------------+
| 16-CoAP:Message ID         | 8543 values of sizes: (16)                    |
|                            | [00010100 01011110](16): 2                    |
|                            | [00010100 01011111](16): 1                    |
|                            | [00010100 01100000](16): 1                    |
|                            | [00010100 01100001](16): 1                    |
|                            | [00010100 01100010](16): 1                    |
|                            | [00010100 01100011](16): 1                    |
|                            | [00010100 01100100](16): 1                    |
|                            | [00010100 01100101](16): 1                    |
|                            | [00010100 01100110](16): 1                    |
|                            | [00010100 01100111](16): 1                    |
|                            | ...                                           |
+----------------------------+-----------------------------------------------+
| 17-CoAP:Token              | 6 values of sizes: (16)                       |
|                            | [11010001 01011001](16): 2951                 |
|                            | [00100001 01010000](16): 2950                 |
|                            | [00110111 00001001](16): 780                  |
|                            | [00011111 00001010](16): 740                  |
|                            | [10110111 00100101](16): 592                  |
|                            | [10001101 01000011](16): 531                  |
+----------------------------+-----------------------------------------------+
| 18-CoAP:Option Delta       | 1 value of size: (4)                          |
|                            | [----0110 ](4): 8544                          |
+----------------------------+-----------------------------------------------+
| 19-CoAP:Option Length      | 2 values of sizes: (4)                        |
|                            | [----0010 ](4): 8334                          |
|                            | [----0001 ](4): 210                           |
+----------------------------+-----------------------------------------------+
| 20-CoAP:Option Value       | 2951 values of sizes: (8, 16)                 |
|                            | [00110100](8): 4                              |
|                            | [00111010](8): 4                              |
|                            | [01100001](8): 4                              |
|                            | [01100100](8): 4                              |
|                            | [01100111](8): 4                              |
|                            | [01101101](8): 4                              |
|                            | [10010001](8): 4                              |
|                            | [10010111](8): 4                              |
|                            | [10101001](8): 4                              |
|                            | [10111110](8): 4                              |
|                            | ...                                           |
+----------------------------+-----------------------------------------------+
| 21-CoAP:Option Delta       | 1 value of size: (4)                          |
|                            | [----0110 ](4): 8544                          |
+----------------------------+-----------------------------------------------+
| 22-CoAP:Option Length      | 2 values of sizes: (4)                        |
|                            | [----0010 ](4): 5901                          |
|                            | [----0001 ](4): 2643                          |
+----------------------------+-----------------------------------------------+
| 23-CoAP:Option Value       | 2 values of sizes: (16, 8)                    |
|                            | [00101101 00010110](16): 5901                 |
|                            | [00111100](8): 2643                           |
+----------------------------+-----------------------------------------------+
| 24-CoAP:Payload Marker     | 1 value of size: (8)                          |
|                            | [11111111](8): 8544                           |
+----------------------------+-----------------------------------------------+
```


Upon inspection, we confirm our observation that several of those fields are constant, *i.e.* `IPv6:Version`, `IPv6:Traffic Class`, `IPv6:Flow Label`, `IPv6:Flow Label`, `IPv6:Next Header`, `IPv6:Hop Limit`, `IPv6:Source Address`, `IPv6:Destination Address`, `UDP:Source Port`, `UDP:Destination Port`, `CoAP:Version`, `CoAP:Token Length`, `CoAP:Code`, `CoAP:Option Delta`, `CoAP:Payload Marker`. A `equal/not-sent` MO/CDA is fit for compressing those fields.

We also observe that some of the fields feature very few different values:

 - `13-CoAP:Type`: 2 values `0b00` and `0b01` for a 2-bits field. 
 - `19-CoAP:Option Length`: 2 values `0b0001`, `0b0010` for a 4-bits length. 
 - `22-CoAP:Option Length`: 2 values `0b0001`, `0b0010` for a 4-bits length. 
 - `23-CoAP:Option Value`: 2 values `0b00101101 00010110` and `0b00111100` of respective lengths 16 and 8

Using microSCHC,

 - `13-CoAP:Type`: the `MSB/LSB` MO/CDA with a TV of length 1 bit, `0b0` yields a 1-bit compression gain.
 - `19-CoAP:Option Length` and `22-CoAP:Option Length`: the `MATCH-MAPPING/MAPPING-SENT` MO/CDA with a TV set to `{[-------0](1): [----0001][4], [-------1](1): [----0010][4]}` yields a 3-bits compression gain.
 - `23-CoAP:Option Value`: the `MATCH-MAPPING/MAPPING-SENT` MO/CDA with a TV set to `{[-------0](1): [00101101 00010110](16), [-------1](1): [00111100](8)}` yields an average compression gain of 12,5 bits.

 Additionnaly, SCHC specifies a specific CDA for fields that can recomputed at the receiver side - in this example: `3-IPv6:Payload Length`, `10-UDP:Length` and `11-UDP:Checksum` - which yields a 3 x 16 bits compression gain.

 

In the following, we define the microSCHC field descriptors for the IPv6 header.

In [1]:
from microschc.protocol.coap import CoAPFields
from microschc.protocol.udp import UDPFields
from microschc import RuleDescriptor, RuleNature, RuleFieldDescriptor, DirectionIndicator as DI, MatchingOperator as MO, CompressionDecompressionAction as CDA, TargetValue, MatchMapping
from microschc.binary import Buffer, Padding
from microschc.protocol.ipv6 import IPv6Fields

ipv6_version_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.VERSION, length=4, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x06', length=4, padding=Padding.LEFT))
ipv6_traffic_class_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.TRAFFIC_CLASS, length=8, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x00', length=8, padding=Padding.LEFT))
ipv6_flow_label_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.FLOW_LABEL, length=20, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x0f\xf8\x5f', length=20, padding=Padding.LEFT))
ipv6_payload_length_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.PAYLOAD_LENGTH, length=16, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.IGNORE, compression_decompression_action=CDA.COMPUTE, target_value=None)
ipv6_next_header_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.NEXT_HEADER, length=8, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x11', length=8, padding=Padding.LEFT))
ipv6_hop_limit_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.HOP_LIMIT, length=8, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x40', length=8, padding=Padding.LEFT))
ipv6_source_address_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.SRC_ADDRESS, length=128, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x20\x01\x0d\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01', length=128, padding=Padding.LEFT))
ipv6_destination_address_fd: RuleFieldDescriptor = RuleFieldDescriptor(id=IPv6Fields.DST_ADDRESS, length=128, position=0, direction=DI.BIDIRECTIONAL, matching_operator=MO.EQUAL, compression_decompression_action=CDA.NOT_SENT, target_value=Buffer(content=b'\x20\x01\x0d\xb8\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02', length=128, padding=Padding.LEFT))


This approach, which is admittedly tedious and verbose, can be simplified using microSCHC templates, as shown below.
The following block yields the same field descriptors as the previous one.

**Note**: the arguments can be provided in multiples ways, either by using a `TargetValue`, *i.e.* `Buffer` or `MatchMapping` or an integer or bytes object. The function `microschc.tools.create_target_value` does the required converions under the hood! 

In [2]:
from microschc.protocol.ipv6 import ipv6_base_header_template

ipv6_field_descriptors = ipv6_base_header_template(
    flow_label=b'\x0f\xf8\x5f',
    src_address=b'\x20\x01\x0d\xb8\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x03',
    dst_address=b'\x20\x01\x0d\xb8\x00\x0a\x00\x00\x00\x00\x00\x00\x00\x00\x00\x20',
    next_header=17,
    hop_limit=64
)

Similarly, we can create the field descriptors for UDP and COAP.

In [3]:
from microschc.protocol.udp import udp_header_template
from microschc.protocol.coap import coap_base_header_template, coap_option_template

udp_field_descriptors = udp_header_template(
    source_port=b'\x90\xa0', # or Buffer(content=b'\x90\xa0', length=16, padding=Padding.LEFT)
    destination_port=b'\x16\x33'  
)

coap_base_header_field_descriptors = coap_base_header_template(
    type=Buffer(content=b'\x00', length=1, padding=Padding.LEFT),
    code=0b01000101,
    token=[b'\xd1\x59', b'\x21\x50', b'\x37\x09', b'\x1f\x0a', b'\xb7\x25', b'\x8d\x43']
)

coap_option_1_field_descriptors = coap_option_template(
    option_delta=b'\x06',
    option_length=[1, 2],
    option_value=None
)
coap_option_2_field_descriptors = coap_option_template(
    option_delta=6,
    option_length=[1,2],
    option_value=[b"\x2d\x16", b"\x3c"]
)

coap_payload_marker_fd = RuleFieldDescriptor(
    id=CoAPFields.PAYLOAD_MARKER, 
    length=8,
    matching_operator=MO.EQUAL,
    compression_decompression_action=CDA.NOT_SENT,
    target_value=Buffer(b'\xff')
)

field_descriptors = (
    ipv6_field_descriptors
    + udp_field_descriptors 
    + coap_base_header_field_descriptors 
    + coap_option_1_field_descriptors 
    + coap_option_2_field_descriptors 
    + [coap_payload_marker_fd]
)

rule_descriptor_0 = RuleDescriptor(
    id=Buffer(content=b'\x00', length=8),
    nature=RuleNature.COMPRESSION,
    field_descriptors=field_descriptors,
)
for fd in field_descriptors:
    print(fd)

{IPv6:Version(4):eq/ns|[----0110 ](4)}
{IPv6:Traffic Class(8):eq/ns|[00000000](8)}
{IPv6:Flow Label(20):eq/ns|[0ff85f](20)}
{IPv6:Payload Length(16):ig/co|None}
{IPv6:Next Header(8):eq/ns|[00010001](8)}
{IPv6:Hop Limit(8):eq/ns|[01000000](8)}
{IPv6:Source Address(128):eq/ns|[20010db8000a00000000000000000003](128)}
{IPv6:Destination Address(128):eq/ns|[20010db8000a00000000000000000020](128)}
{UDP:Source Port(16):eq/ns|[10010000 10100000](16)}
{UDP:Destination Port(16):eq/ns|[00010110 00110011](16)}
{UDP:Length(16):ig/co|None}
{UDP:Checksum(16):ig/co|None}
{CoAP:Version(2):eq/ns|[------01 ](2)}
{CoAP:Type(2):ms/ls|[-------0 ](1)}
{CoAP:Token Length(4):eq/ns|[----0010 ](4)}
{CoAP:Code(8):eq/ns|[01000101](8)}
{CoAP:Message ID(16):ig/vs|None}
{CoAP:Token(16):ma/ms|{[-----000 ](3):[11010001 01011001](16),[-----001 ](3):[00100001 01010000](16),[-----010 ](3):[00110111 00001001](16),[-----011 ](3):[00011111 00001010](16),[-----100 ](3):[10110111 00100101](16),[-----101 ](3):[10001101 01000011]

Now that a rule is defined for the compresssion of the first template, we create a context and a context manager.

In this example, the context contains two rules: one for compression and one for no compression.
The context attribute `parser_id` is set to the ID of the parser and is used by the context manager to instantiate the correct parser.
The other context attributes, *i.e.* `id`, `description`, `interface_id` are informational and has no functional use in microschc.


In [4]:
from microschc import ContextManager, Context, Stack
no_compression_rule_id: Buffer = Buffer(content=b'\x01', length=8)
no_compression_rule_descriptor: RuleDescriptor = RuleDescriptor(id=no_compression_rule_id, nature=RuleNature.NO_COMPRESSION)

context = Context(
    id="quickstart-context", # id for the context, can be any string. Primary use is to identify the context in json files
    description="Context for IoT device communication", # description of the context, can be any string. Primiary use is to describe the context in json files
    interface_id="default", # interface ID for the context, can be any string. Interface on which the context is used.
    parser_id=Stack.IPV6_UDP_COAP, # parser ID for the context, can be any of the parsers defined in microschc.protocol or a complete stack.
    ruleset=[rule_descriptor_0, no_compression_rule_descriptor],
)
context_manager: ContextManager = ContextManager(context=context)

Let's use the context manager to compress a packet matched by the compression rule.

In [5]:
from typing import List
from microschc.extras.io.pcapng import packets_list
from microschc.rfc8724 import PacketDescriptor
packets: List[Buffer] = packets_list("./quickstart-files/leshan-thermostat-readings.pcapng")
for packet in packets[100:101]:
    print(f"Packet: {packet}")
    packet_descriptor: PacketDescriptor = context_manager.parser.parse(packet)
    for i, field in enumerate(packet_descriptor.fields):
        print(f"{i}-{field.id}: {field.value}")
    # Compress the packet
    compressed_packet = context_manager.compress(packet)
    print("Compressed Packet: ", compressed_packet)
    # Decompress the packet
    decompressed_packet = context_manager.decompress(compressed_packet)
    print("Decompressed Packet: ", decompressed_packet)
    if packet != decompressed_packet:
        raise ValueError("Decompressed packet does not match original packet")
    else:
        print("Decompressed packet matches original packet")

Packet: [600ff85f001c114020010db8000a0000000000000000000320010db8000a0000000000000000002090a01633001c2923524514b537096176613cfffb4031666666666666](544)
0-IPv6:Version: [----0110 ](4)
1-IPv6:Traffic Class: [00000000](8)
2-IPv6:Flow Label: [0ff85f](20)
3-IPv6:Payload Length: [00000000 00011100](16)
4-IPv6:Next Header: [00010001](8)
5-IPv6:Hop Limit: [01000000](8)
6-IPv6:Source Address: [20010db8000a00000000000000000003](128)
7-IPv6:Destination Address: [20010db8000a00000000000000000020](128)
8-UDP:Source Port: [10010000 10100000](16)
9-UDP:Destination Port: [00010110 00110011](16)
10-UDP:Length: [00000000 00011100](16)
11-UDP:Checksum: [00101001 00100011](16)
12-CoAP:Version: [------01 ](2)
13-CoAP:Type: [------01 ](2)
14-CoAP:Token Length: [----0010 ](4)
15-CoAP:Code: [01000101](8)
16-CoAP:Message ID: [00010100 10110101](16)
17-CoAP:Token: [00110111 00001001](16)
18-CoAP:Option Delta: [----0110 ](4)
19-CoAP:Option Length: [----0001 ](4)
20-CoAP:Option Value: [01110110](8)
21-CoAP:Option

Contexts can be stored in and restored from JSON files. This is achieved using `Context.json()` and `Context.from_json()` functions.

In [6]:
context_json: str = context.json()
duplicate_context = Context.from_json(context_json)

# The duplicate context should be identical to the original one
print(duplicate_context.ruleset)

[[[00000000](8)](25) {IPv6:Version(4):eq/ns|[----0110 ](4)}|{IPv6:Traffic Class(8):eq/ns|[00000000](8)}|{IPv6:Flow Label(20):eq/ns|[0ff85f](20)}|{IPv6:Payload Length(16):ig/co|None}|{IPv6:Next Header(8):eq/ns|[00010001](8)}|{IPv6:Hop Limit(8):eq/ns|[01000000](8)}|{IPv6:Source Address(128):eq/ns|[20010db8000a00000000000000000003](128)}|{IPv6:Destination Address(128):eq/ns|[20010db8000a00000000000000000020](128)}|{UDP:Source Port(16):eq/ns|[10010000 10100000](16)}|{UDP:Destination Port(16):eq/ns|[00010110 00110011](16)}|{UDP:Length(16):ig/co|None}|{UDP:Checksum(16):ig/co|None}|{CoAP:Version(2):eq/ns|[------01 ](2)}|{CoAP:Type(2):ms/ls|[-------0 ](1)}|{CoAP:Token Length(4):eq/ns|[----0010 ](4)}|{CoAP:Code(8):eq/ns|[01000101](8)}|{CoAP:Message ID(16):ig/vs|None}|{CoAP:Token(16):ma/ms|{[-----000 ](3):[11010001 01011001](16),[-----001 ](3):[00100001 01010000](16),[-----010 ](3):[00110111 00001001](16),[-----011 ](3):[00011111 00001010](16),[-----100 ](3):[10110111 00100101](16),[-----101 ](3

### Extras: Evaluation of a SCHC context

If your environment features `python-pcapng`, microSCHC can evaluate a SCHC context from a pcap-ng file. In the following, we will load a predefined context and evaluate it on a PCAPng dataset.

In [7]:
from microschc.extras.io.pcapng import packets_list

with open(file='./quickstart-files/syntactic.json', mode='r') as f:
    json_str: str = f.read()

context: Context = Context.from_json(json_str=json_str)
packets = packets_list(filepath='./quickstart-files/leshan-thermostat-readings.pcapng')

context_manager = ContextManager(context=context)

uncompressed_total = 0
compressed_total = 0

for packet in packets:
    uncompressed_total += packet.length
    compressed_packet = context_manager.compress(packet)
    compressed_total += compressed_packet.length
    uncompressed_packet = context_manager.decompress(compressed_packet)
    if uncompressed_packet != packet:
        raise ValueError(f"uncompressed packet different from original")
    

print(f'compression ratio: {uncompressed_total/compressed_total}')
    

compression ratio: 4.107287830020684


As shown in the previous cell, using the context provided, we've achieved a compression ratio of 4.1, meaning we've divided the original size of the traffic by 4!